# Librerias

In [ ]:
import pandas as pd
import os
from pathlib import Path
import sys
import re

# Añadir la carpeta raíz del proyecto a sys.path
sys.path.append(os.path.abspath("c:/Users/osmarrincon/Documents/capresoca-data-automation"))
#sys.path.append(os.path.abspath("D:\Proyectos Python\capresoca-data-automation"))

from src.file_loader import cargar_maestros_ADRES

# rutas

In [ ]:
R_bd = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\informes\2025\CTO135.2025 Informe  #9\ACTIVIDAD 14\Estelita\EXCEL.xlsx"

R_MS_SIE = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\SIE\Aseguramiento\ms_sie\Reporte_Validación Archivos Maestro_2025_08_22.csv"
R_MS_EPS025 = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Maestro\MS\2025-02\EPS025MS0020082025.TXT"
R_MS_EPSC25 = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Contributivo\Maestro\2025-2\EPSC25MC0020082025.TXT"
R_pila_3047 = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Contributivo\Compensación\Pila consiliada ADRES\Pila_Unificado_Con_Aportante_2018_2025.TXT"

# Dataframes

In [ ]:
# Cargar y combinar los maestros
maestro_ADRES = cargar_maestros_ADRES(R_MS_EPS025, R_MS_EPSC25)
# Carga Maestro SIE
df_sie = pd.read_csv(R_MS_SIE, sep=';', dtype=str, header=0, encoding='ANSI')
Pila_3047 = pd.read_csv(R_pila_3047, sep=',', dtype=str, header=0, encoding='UTF-16')

# Base de datos Excel 
df_bd = pd.read_excel(R_bd, sheet_name="Hoja1", header=0, dtype=str)

# Limpiar datos

In [ ]:
Pila_3047.columns

In [ ]:
# Contar las combinaciones más frecuentes de Tipo_doc_cotizante y doc_cotizante
mas_repetido = Pila_3047.groupby(['Tipo_doc_cotizante', 'doc_cotizante']).size().reset_index(name='conteo').sort_values('conteo', ascending=False).iloc[0]

# Filtrar el dataframe para mostrar todos los registros con ese id
registro_mas_repetido = Pila_3047[
    (Pila_3047['Tipo_doc_cotizante'] == mas_repetido['Tipo_doc_cotizante']) &
    (Pila_3047['doc_cotizante'] == mas_repetido['doc_cotizante'])
]

In [ ]:
# Convertir 'perido_pago_del_aportante' a datetime para comparar fechas
Pila_3047['perido_pago_del_aportante'] = pd.to_datetime(Pila_3047['perido_pago_del_aportante'], format='%Y-%m')

# Ordenar por identificadores y fecha descendente
Pila_3047 = Pila_3047.sort_values(['Tipo_doc_cotizante', 'doc_cotizante', 'perido_pago_del_aportante'], ascending=[True, True, False])

# Dejar solo el registro más reciente por cada id
Pila_3047 = Pila_3047.drop_duplicates(subset=['Tipo_doc_cotizante', 'doc_cotizante'], keep='first')

# Si quieres mantener el formato original de la fecha:
Pila_3047['perido_pago_del_aportante'] = Pila_3047['perido_pago_del_aportante'].dt.strftime('%Y-%m')

In [ ]:
print(Pila_3047[(Pila_3047['Tipo_doc_cotizante'] == 'CC') & (Pila_3047['doc_cotizante'] == '1007539785')])

# Cruce base de datos

In [ ]:
df_bd.columns

In [ ]:
Pila_3047.columns

In [ ]:
# Crear un diccionario para mapear los valores desde Pila_3047
map_razon = dict(zip(Pila_3047['doc_cotizante'], Pila_3047['Razon_Soacial']))
map_ibc = dict(zip(Pila_3047['doc_cotizante'], Pila_3047['ibc']))

# Asignar los valores al dataframe df_bd
df_bd['Nombre de empleador o  contratante'] = df_bd['Número Documento'].map(map_razon)
df_bd['Ingreso Base de Cotización'] = df_bd['Número Documento'].map(map_ibc)

In [ ]:
# Hacer un merge temporal para transferir los datos de contacto de df_sie a df_bd
df_bd = df_bd.merge(
    df_sie[['numero_identificacion', 'celular', 'direccion', 'municipio', 'correo_electronico']],
    how='left',
    left_on='Número Documento',
    right_on='numero_identificacion'
)

# Asignar los valores a las columnas correspondientes en df_bd
df_bd['Teléfono'] = df_bd['celular']
df_bd['Dirección de Residencia'] = df_bd['direccion']
df_bd['Ciudad'] = df_bd['municipio']
df_bd['Correo electrónico'] = df_bd['correo_electronico']

# Eliminar columnas auxiliares del merge
df_bd = df_bd.drop(columns=['numero_identificacion', 'celular', 'direccion', 'municipio', 'correo_electronico'])

In [ ]:
# Crear un diccionario para mapear los valores desde maestro_ADRES
map_estado = dict(zip(maestro_ADRES['HST_IDN_NUMERO_IDENTIFICACION'], maestro_ADRES['TPS_EST_AFL_ID']))
map_eps = dict(zip(maestro_ADRES['HST_IDN_NUMERO_IDENTIFICACION'], maestro_ADRES['ENT_ID']))

# Asignar los valores al dataframe df_bd
df_bd['Estado de Afiliación'] = df_bd['Número Documento'].map(map_estado).fillna('Registro no existe o en otra EPS')
df_bd['EPS'] = df_bd['Número Documento'].map(map_eps).fillna('Registro no existe o en otra EPS')

In [ ]:
with pd.ExcelWriter(R_bd, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_bd.to_excel(writer, sheet_name='Df_Resultado', index=False)